### Segmenting & Clustering Toronto Neighborhood

In [110]:
#Import required libraries
import numpy as np
import pandas as pd

Read the wikipedia file on Canada postal codes and neighborhood using Pandas

In [111]:
path = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [112]:
df = pd.read_html(path)[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [113]:
# clean up the dataframe to remove the rows with Borough not assigned
df = df[df['Borough'] != 'Not assigned']

For postal codes with more than one neighborhood, split the neighborhood separated by '/' and make it a list separated by commas

In [114]:
df["Neighborhood"] = df["Neighborhood"].str.split("/", expand = False)

In [115]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,[Parkwoods]
3,M4A,North York,[Victoria Village]
4,M5A,Downtown Toronto,"[Regent Park , Harbourfront]"
5,M6A,North York,"[Lawrence Manor , Lawrence Heights]"
6,M7A,Downtown Toronto,"[Queen's Park , Ontario Provincial Government]"


In [79]:
#check if there is any row with neighborhood "Not assigned" or "NaN"

In [116]:
(df['Neighborhood'] == 'Not assigned').value_counts()

False    103
Name: Neighborhood, dtype: int64

In [117]:
(df['Neighborhood'] == 'NaN').value_counts()

False    103
Name: Neighborhood, dtype: int64

In [119]:
#display the shape
df.shape

(103, 3)